In [ ]:
from urllib import request
from bs4 import BeautifulSoup
import re
# http://blog.csdn.net/liu5257/article/details/53740214
'''
os模块负责程序与操作系统的交互，提供了访问操作系统底层的接口;
sys模块负责程序与python解释器的交互，提供了一系列的函数和变量，用于操控python的运行时环境
'''
import sys

In [ ]:
if __name__ == '__main__':
    # 创建txt文件
    file = open('一念永恒.txt', 'w', encoding='utf-8')
    # 目录
    target_url = 'http://www.biqukan.com/1_1094/'
    head = {}
    head['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
    # 构建请求
    target_req = request.Request(url=target_url, headers = head)
    # 接收响应
    target_response = request.urlopen(target_req)
    # 解码处理
    # ignore -> 忽略非法字符
    target_html = target_response.read().decode('gbk', 'ignore')
    # 创建BeautifulSoup对象
    listmain_soup = BeautifulSoup(target_html, 'lxml')
    # 搜索文档树,找出div标签中class为listmain的所有子标签
    chapters = listmain_soup.find_all('div',class_ = 'listmain')
    # 将查询结果在创建一个BeautifulSoup对象继续进行解析
    download_soup= BeautifulSoup(str(chapters), 'lxml')
    # 计算章节个数
    numbers = (len(download_soup.dl.contents) - 1) / 2 - 8
    index = 1
    # 记录章节标志位，只要正文卷下面的链接，最新章节列表的链接舍弃
    begin_flag = False
    # 遍历文档树下dl标签中的所有子节点
    for child in download_soup.dl.children:
        # 滤除回车
        if child != '\n':
            # 定位到正文卷
            if child.string == u"《一念永恒》正文卷":
                begin_flag = True
            # 爬取链接并下载内容
            if begin_flag == True and child.a != None:
                download_url = 'http://www.biqukan.com' + child.a.get('href')
                # 构建请求
                download_req = request.Request(url = download_url, headers = head)
                # 接收响应
                download_response = request.urlopen(download_req)
                # 解码并忽略非法字符
                download_html = download_response.read().decode('gbk', 'ignore')
                # 章节名
                download_name = child.string
                # 创建BeautifulSoup对象
                soup_texts = BeautifulSoup(download_html, 'lxml')
                # 找到内容标签
                texts = soup_texts.find_all(id = 'content', class_ = 'showtxt')
                # 创建BeautifulSoup对象
                soup_text = BeautifulSoup(str(texts), 'lxml')
                write_flag = True
                # 写章节名
                file.write(download_name + '\n\n')
                # 将爬取内容写入文件
                # 处理特殊字符
                for each in soup_text.div.text.replace('\xa0', ''):
                    if each == 'h':
                        write_flag = False
                    if write_flag == True and each != ' ':
                        file.write(each)
                    if write_flag == True and each == '\r':
                        file.write('\n')
                file.write('\n\n')
                # 打印爬虫进度
                sys.stdout.write("已下载: %.3f%%" % float(index/numbers) + '\r')
                sys.stdout.flush()
                index += 1
    file.close()
                    